# 01. Let's import necessary libraries

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
import swatp_pst
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

In [ ]:
swatp_pst.__version__

# 00. provide infos

In [ ]:
#working directory for mac
prj_dir = "//Users/seonggyu.park/Documents/projects"
swatp_wd = "/Users/seonggyu.park/Documents/projects/TxtInOut"

In [ ]:
# working directory for win
# prj_dir = "D:\\jj\\opt_2nd"
# swatp_wd = "D:\\spark\\jj\\TxtInOut_Imsil_rye_rot_r2"

In [ ]:
# calibration period
cal_start = '1/1/2000'
cal_end = '12/31/2001'
# time step
time_step = 'month'
# locations (what our targets)
chs = [106]
grids = [26989]
obd_file = "stf_mon.obd.csv"
obd_colnam = "cha106"
mfObdFile = "gwl_day.obd.csv"
mfObdColnam = "gwl26989"

# 01. inital setup

In [ ]:
from swatp_pst import handler

In [ ]:
# copy all necessary files (exes) to your working direcotry
handler.init_setup(prj_dir, swatp_wd)

In [ ]:
# create swatp_con file
handler.create_swatp_pst_con(
    prj_dir, swatp_wd, cal_start, cal_end, chs, grids, time_step=time_step)

In [ ]:
os.getcwd()

In [ ]:
wd = os.getcwd()

### Let's do initial run!

In [ ]:
# you need to run the model first if outputs do not exit.
# pyemu.os_utils.run(r"swatplus.exe", cwd=".")

# 02. Build template files

In [ ]:
from swatp_pst.pst_utils import PstUtil

In [ ]:
m1 = PstUtil(os.path.join(prj_dir, "main_opt"))

In [ ]:
m1.cal_to_tpl_file()

In [ ]:
m1.gw_input_to_tpl()

# 03. Build instruction files

In [ ]:
# extract output first
# m1 = SWATpOut(wd)
m1.extract_mon_stf(chs, cal_start, cal_end)

In [ ]:
m1.stf_obd_to_ins?

In [ ]:
m1.stf_obd_to_ins('stf_106.txt', obd_file, obd_colnam, cal_start, cal_end)

In [ ]:

m1.mf_obd_to_ins("gwl_26989.txt", mfObdColnam, cal_start, cal_end)

# 04. Create PEST control file

In [ ]:
os.getcwd()

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)

In [ ]:
par = pst.parameter_data
par

In [ ]:
obd = pst.observation_data
obd

## 05.02 Adjust initial parameter values and their ranges

In [ ]:
par = pst.parameter_data

In [ ]:
par

## 05.03 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 05.04 Provide actual observed values to control file

In [ ]:
os.getcwd()

In [ ]:
# Streamflow
stf_obd = pd.read_csv(obd_file,
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# groundwater level
gwl_obd = pd.read_csv(mfObdFile,
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
gwl_obd = gwl_obd[cal_start: cal_end]
gwl_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
tot_obd = []

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += gwl_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
tot_obd

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 06. Create the control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [ ]:
pst.control_data.formatted_values

In [ ]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

- ### add new PEST++ variables like so:

In [ ]:
# check the dictionary again
pst.pestpp_options

In [ ]:
pst.write('swatp_dummy.pst', version=2)

- ### You can also read and load the existing pest control file.

In [ ]:
# pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
# epst_read.parameter_data